# Training MNIST Model

The goal of this is to create a larger-than-necessary model for classifying handwritten MNIST digits. After training, the model will be serialized in a binary format to later be run using **raw** CUDA and OpenCL.

In [5]:
# required imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader 
from torchvision.datasets import MNIST
import numpy as np

## seed for reproducibility
torch.manual_seed(0)

In [6]:
# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10) # 0 - 9 digits
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleNN()

In [7]:
# Hyperparameters
batch_size = 64
learning_rate = 0.001
epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# data loading
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Datasets and DataLoaders
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
# training the model
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

Epoch 1/5, Loss: 0.40121362450471054
Epoch 2/5, Loss: 0.19369668023053135
Epoch 3/5, Loss: 0.14247536396425067
Epoch 4/5, Loss: 0.11517478759958545
Epoch 5/5, Loss: 0.09768994483299481


In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the model on the test images: {100 * correct / total}%')

Accuracy of the model on the test images: 96.52%


In [11]:
# rather sloppy section to save model weights in a binary format
# somewhat difficult for some reason

import glob
import os

os.system('rm *.bin')

0

In [12]:
# checking the distribution of weights

# this comes after seeing the bill daily talk where weights are: normally distributed **before** pruning and non-normally distributed after pruning
i = 0
for layer in model.children():
    if isinstance(layer, nn.Linear):
        state = layer.state_dict()
        state['weight'].flatten().data.cpu().numpy().tofile(f'weights{i}.bin')
        state['bias'].flatten().data.cpu().numpy().tofile(f'bias{i}.bin')
        i += 1

In [13]:
# appending all the model weights to a single binary file
with open('model', 'wb') as f:
    for file in glob.glob('*.bin'):
        with open(file, 'rb') as f2:
            f.write(f2.read())

os.system('rm *.bin')

0

## Saving MNIST Outputs

Save the MNIST and model outputs as a position-dependent pair.

In [14]:
transform = transforms.Compose([transforms.ToTensor()])
mnist_train = MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = MNIST(root='./data', train=False, download=True, transform=transform)

def flatten_and_convert_to_numpy(dataset):
    images = []
    labels = []
    for img, label in dataset:
        images.append(img.view(-1).numpy())
        labels.append(label)
    return np.array(images), np.array(labels)

train_images, train_labels = flatten_and_convert_to_numpy(mnist_train)
test_images, test_labels = flatten_and_convert_to_numpy(mnist_test)

# Step 4: Save the flattened data in a binary format
train_images.tofile('../cuda/mnist_train_images.bin')
train_labels.tofile('../cuda/mnist_train_labels.bin')
test_images.tofile('../cuda/mnist_test_images.bin')
test_labels.tofile('../cuda/mnist_test_labels.bin')

In [15]:
len(train_images), len(train_labels), len(test_images), len(test_labels)

(60000, 60000, 10000, 10000)

# Generating Outputs

This section seeks to match outputs from PyTorch to real data.

In [20]:
# generating the sample outputs
model.eval()
with torch.no_grad(), open('../cuda/sample_outputs.bin', 'wb') as f:
    for image in train_images:
        output = model(torch.tensor(image).float())
        with open("tmp_output.bin", "wb") as f2:
            output.numpy().tofile(f2)
        # the size of tmp_output is 40 bytes
        assert os.path.getsize("tmp_output.bin") == 40
        with open("tmp_output.bin", "rb") as f2:
            f.write(f2.read())